# Resultados de la Penca Rusia 2018 - Argos Universitario

In [1]:
from IPython.display import Image
import pandas as pd
from os import walk
%pylab inline
import re
import matplotlib.pyplot as plt
plt.rc('legend', fontsize='small')
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import display, HTML
from datetime import timedelta
from __future__ import print_function, division
import statsmodels.formula.api as smf
import pylab as pl
pd.set_option('precision', 2)

#import plotly.plotly as py
#import plotly.graph_objs as go
#from plotly.tools import FigureFactory as FF
#import scipy
#from xlrd import open_workbook

Populating the interactive namespace from numpy and matplotlib


In [2]:
dire="files/"

def points(xg1,xg2,rg1,rg2):
  res = 0
  if not math.isnan(xg1) and not math.isnan(rg1) and xg1==rg1:
    res += 1
  if not math.isnan(xg2) and not math.isnan(rg2) and xg2==rg2:
    res += 1
  if not math.isnan(xg1) and not math.isnan(rg1):
    if not math.isnan(xg2) and not math.isnan(rg2):
      if ((xg1-xg2) < 0 and (rg1-rg2)< 0 ):
        res+=3 
      if ((xg1-xg2) > 0 and (rg1-rg2)> 0 ):
        res+=3 
      if ((xg1-xg2) == 0 and (rg1-rg2)== 0 ):
        res+=3
  return res

def seller(x):
  return x.split('_')[0].replace('Penca Argos Rusia 2018', '').replace('-','').replace(' ','')

def name(x):
  return x.split('_')[1].replace('Penca Argos Rusia 2018', '').replace('-','').replace(' ','')

def name2(x):
  return x.split('|')[1]


def one():
  return 1

def incomplete(xg1,xg2):
  if math.isnan(xg1) and math.isnan(xg2):
    return 2
  if math.isnan(xg1):
    return 1
  if math.isnan(xg2):
    return 1

def incompleteff(label,value):
  if not label and math.isnan(value):
    return 2
  if not label:
    return 1
  if math.isnan(value):
    return 1


def buildx(file):
  xls = pd.ExcelFile(dire+file)
  y = pd.read_excel(xls, 'Fase de grupos')
  name=y[y.columns[4]][0]
  mail=y[y.columns[10]][0]
  #print(y[y.columns[10]][0])
   
  y=y[[y.columns[6],y.columns[7],y.columns[8],y.columns[9],]]
  y.columns = ['t1','g1','g2','t2']
  y['id']="%s|%s|%s" % (file, name, mail)
  y['name']=name
  #select rows
  A=y.iloc[4:10]
  B=y.iloc[14:20]
  C=y.iloc[24:30]
  D=y.iloc[34:40]
  E=y.iloc[44:50]
  F=y.iloc[54:60]
  G=y.iloc[64:70]
  H=y.iloc[74:80]
 
  #ff = pd.read_excel(xls, 'Fase final')
  #oct_etq=ff.iloc[4:10]
  
 
  z = pd.concat([A,B,C,D,E,F,G,H],axis=0)
  


  return pd.DataFrame(z)

def buildff(file):
  xls = pd.ExcelFile(dire+file)

  y = pd.read_excel(xls, 'Fase de grupos')
  name=y[y.columns[4]][0]
  mail=y[y.columns[4]][0]
  y = pd.read_excel(xls, 'Fase final')
  
  #name=y[y.columns[4]][0]
  #mail=y[y.columns[4]][0]
  y=y[[y.columns[3],y.columns[4],]]
  y.columns = ['label','value']
  oct1=y.iloc[4:6]
  oct2=y.iloc[7:9]
  oct3=y.iloc[11:13]
  oct4=y.iloc[14:16]
  oct5=y.iloc[18:20]
  oct6=y.iloc[21:23]
  oct7=y.iloc[25:27]
  oct8=y.iloc[28:30]



  #z = pd.concat([A,B,C,D,E,F,G,H],axis=0)
  z = pd.concat([oct1,oct2,oct3,oct4,oct5,oct6,oct7,oct8],axis=0)
  
    
  z['id']="%s|%s|%s" % (file, name, mail)
  return pd.DataFrame(z)  

    
r=buildx("RESULTADO_RESULTADO.xlsx")
x=pd.DataFrame() #fase grupos
xff=pd.DataFrame() #fase final
for file in filter(lambda k: k.endswith('.xlsx') or k.endswith('.xls'), next(walk(dire))[2][:]):
  z=buildx(file)
  ff=buildff(file)  
  z = pd.concat([z,r],axis=1) 
  z.columns = ['xt1','xg1','xg2','xt2','idx','namex','rt1','rg1','rg2','rt2','idr','namer']  
  x = x.append(z)
  xff = xff.append(ff)  

x['points'] =  pd.Series(x.apply(lambda x:points(x.xg1,x.xg2,x.rg1,x.rg2),axis = 1))
x['incompletes'] =  pd.Series(x.apply(lambda x:incomplete(x.xg1,x.xg2),axis = 1))
xff['incompletes'] =  pd.Series(xff.apply(lambda x:incompleteff(x.label,x.value),axis = 1))


tabla = x.groupby(['idx']).sum().reset_index()
tablaff = xff.groupby(['id']).sum().reset_index()

tabla['seller'] =  pd.Series(tabla.apply(lambda tabla:seller(tabla.idx),axis = 1))
tabla['name'] =  pd.Series(tabla.apply(lambda tabla:name(tabla.idx),axis = 1))
tabla['name2'] =  pd.Series(tabla.apply(lambda tabla:name2(tabla.idx),axis = 1))
tabla['one'] =  pd.Series(tabla.apply(lambda tabla:one(),axis = 1))
tablaff['seller'] =  pd.Series(tabla.apply(lambda tabla:seller(tabla.idx),axis = 1))
tablaff['one'] =  pd.Series(tabla.apply(lambda tabla:one(),axis = 1))



tabla1 = tabla.sort_values(['incompletes'], ascending=False)[['seller','points','incompletes','idx']]
tablaff1 = tablaff.sort_values(['incompletes'], ascending=False)[['seller','incompletes','id']]

display (tablaff1.head())

print ('total='+str(tabla1.shape[0]))

display(tabla1[tabla1['seller']=='JonathanMurana'])    #
display (tabla1.head())    

tabla2 = tabla.sort_values(['points'], ascending=False)[['idx','name2','seller','points']]

tabla2=tabla2.reset_index()
display(tabla2[tabla2['seller']=='JonathanMurana'])    #


#tabla2['pos']=tabla2.index


#display (tabla2) 





ranking = tabla.groupby(['seller']).sum().reset_index()
ranking = ranking.sort_values(['one'], ascending=False)[['seller','one']]
print(ranking)
print("ultimo partido Brasil-Suiza")
tabla2.style



,seller,incompletes,id
250,RESULTADO,16.0,RESULTADO_RESULTADO.xlsx|aaaaaa|aaaaaa
0,FrankoGraziano,0.0,FrankoGraziano_21_EduardoMartinez .xlsx|eduard...
176,PaoloDeFranco,0.0,Paolo De Franco_ Sergio Vitola.xlsx|SERGIO VIT...
163,Pancho,0.0,Pancho_Fede Porto de Penca Argos Rusia 2018.xl...
164,Pancho,0.0,Pancho_FedeDoreste.xlsx|Federico Doreste|Feder...


total=257


,seller,points,incompletes,idx
43,JonathanMurana,16,0.0,JonathanMurana_CeciliaTechera.xlsx|Cecilia Tec...
44,JonathanMurana,17,0.0,JonathanMurana_GustavoGonzalez.xlsx|Gustavo Go...
45,JonathanMurana,15,0.0,JonathanMurana_JimenaMuraña.xlsx|Jimena Muraña...
46,JonathanMurana,15,0.0,JonathanMurana_JonathanMurana.xlsx|Jonathan Mu...
47,JonathanMurana,19,0.0,JonathanMurana_JuanAngel.xlsx|Juan Angel Álvar...
48,JonathanMurana,14,0.0,JonathanMurana_LazaroPereira.xlsx|Lazaro Perei...
49,JonathanMurana,19,0.0,JonathanMurana_MarcosValdivia.xlsx|Marcos Vald...
50,JonathanMurana,17,0.0,JonathanMurana_MartinViurrarena.xlsx|Martin Vi...
51,JonathanMurana,25,0.0,JonathanMurana_MauricioPirene.xlsx|Mauricio Pi...
52,JonathanMurana,15,0.0,JonathanMurana_NicolasBillar.xlsx|Nicolás Bill...


,seller,points,incompletes,idx
250,RESULTADO,55,74.0,RESULTADO_RESULTADO.xlsx|aaaaaa|q@asd.com
0,FrankoGraziano,17,0.0,FrankoGraziano_21_EduardoMartinez .xlsx|eduard...
176,PaoloDeFranco,16,0.0,Paolo De Franco_ Sergio Vitola.xlsx|SERGIO VIT...
163,Pancho,19,0.0,Pancho_Fede Porto de Penca Argos Rusia 2018.xl...
164,Pancho,18,0.0,Pancho_FedeDoreste.xlsx|Federico Doreste|fede....


,index,idx,name2,seller,points
19,51,JonathanMurana_MauricioPirene.xlsx|Mauricio Pi...,Mauricio Pirené,JonathanMurana,25
38,53,JonathanMurana_NuberMuraña.xlsx|Nuber Muraña|j...,Nuber Muraña,JonathanMurana,23
58,56,JonathanMurana_TatianaAlvarez.xlsx|Tatiana Álv...,Tatiana Álvarez,JonathanMurana,22
112,49,JonathanMurana_MarcosValdivia.xlsx|Marcos Vald...,Marcos Valdivia,JonathanMurana,19
121,47,JonathanMurana_JuanAngel.xlsx|Juan Angel Álvar...,Juan Angel Álvarez,JonathanMurana,19
149,44,JonathanMurana_GustavoGonzalez.xlsx|Gustavo Go...,Gustavo Gonzalez,JonathanMurana,17
159,50,JonathanMurana_MartinViurrarena.xlsx|Martin Vi...,Martin Viurrarena,JonathanMurana,17
178,43,JonathanMurana_CeciliaTechera.xlsx|Cecilia Tec...,Cecilia Techera,JonathanMurana,16
196,52,JonathanMurana_NicolasBillar.xlsx|Nicolás Bill...,Nicolás Billar,JonathanMurana,15
206,45,JonathanMurana_JimenaMuraña.xlsx|Jimena Muraña...,Jimena Muraña,JonathanMurana,15


                 seller  one
5       JavierHernández   30
12         NicolásFripp   28
7          LucasBouissa   26
3        FrankoGraziano   21
2   FedericoArmandPilon   19
11                Mauro   16
14               Pancho   14
6        JonathanMurana   14
4                Galvao   12
15        PaoloDeFranco   12
8       MarianoMartinez   11
9           MateoGonnet   10
1           ErnestoRoca   10
10     MathiasLegarreta   10
0          AgustinPerez    9
18     SebastianAlvarez    6
16                 Pepe    5
13         PabloGronros    3
17            RESULTADO    1
ultimo partido Brasil-Suiza


,index,idx,name2,seller,points
0,250,RESULTADO_RESULTADO.xlsx|aaaaaa|q@asd.com,aaaaaa,RESULTADO,55
1,255,SebastianAlvarez_5_GustavoParma.xlsx|Gustavo Parma|gparma@coca-cola.com.uy,Gustavo Parma,SebastianAlvarez,33
2,178,Paolo De Franco_Cecilia Cristiano.xlsx|Cecilia Cristiano|nan,Cecilia Cristiano,PaoloDeFranco,30
3,70,LucasBouissa_22_JairCoceres.xlsx|Jair Coceres|jair.coceres@gmail.com,Jair Coceres,LucasBouissa,29
4,82,LucasBouissa_9_Sclei.xlsx|Diego Scleidorovich|sclei@vera.com.uy,Diego Scleidorovich,LucasBouissa,28
5,106,Mathias Legarreta_02.xlsx|Mathias Legarreta|mathias.legarreta@nutrapet.com.uy,Mathias Legarreta,MathiasLegarreta,28
6,179,Paolo De Franco_Ernesto Ruting.xlsx|Ernesto RUTING|Ernesto.ruting@brou.com.uy,Ernesto RUTING,PaoloDeFranco,28
7,22,Javier Hernández_19_Mauricio Stagno.xlsx|Mauricio Stagno|stagnomauricio@gmail.com,Mauricio Stagno,JavierHernández,28
8,219,Penca Argos Rusia 2018 - FrankoGraziano_13_Gonzalo Keulyan.xlsx|Gonzalo keuylian|gonzalokeuylian@gmail.com,Gonzalo keuylian,FrankoGraziano,27
9,57,LucasBouissa_10_NicolasImbriaco.xlsx|Nicolas Imbriaco|nicolasimbraco@hotmail.com,Nicolas Imbriaco,LucasBouissa,27
